In [1]:
# magics to auto-reload external modules in case you are making changes to langchain while working on this notebook
%load_ext autoreload
%autoreload 2

In [2]:
# get a token: https://replicate.com/account

from getpass import getpass

REPLICATE_API_TOKEN = getpass()

In [3]:
import os

os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [4]:
from langchain.chains import LLMChain
from langchain_community.llms import Replicate
from langchain_core.prompts import PromptTemplate

In [8]:

llm = Replicate(
    model="andreasjansson/llama-2-13b-chat-gguf:60ec5dda9ff9ee0b6f786c9d1157842e6ab3cc931139ad98fe99e08a35c5d4d4",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1, "jsonschema": "{\n \"type\": \"object\", \n \"properties\": {\n \"Id\" : {  \"type\" : \"integer\"}, \n \"name\" : {\n \"type\" : \"string\"}, \n \"age\" : {\n \"type\" : \"integer\"}, \n \"gender\" : {\n \"type\" : \"string\" }, \n \"diagnosis\" : {\n \"type\" : \"string\" }} }"},
)
prompt = """"    
System: You are an AI that summarizes medical conversations into a structured JSON format. Given the medical transcript below, provide a summary by extracting key-value pairs. Only use the information explicitly mentioned in the transcript, and you must not infer or assume any details that are not directly stated.
Speaker 1: Good morning, Nurse. Could you please update me on the status of our patient, Mr. Anderson?
Speaker 2: Good morning, Doctor. Mr. Anderson's blood pressure has stabilized at 100/70. His blood glucose level this morning was 8 mmol/L.
Speaker 1: That's good to hear. Has he taken his medication for the morning?
Speaker 2: Yes, Doctor. He took his Metformin at 7 AM, as prescribed.
Speaker 1: Excellent. Please continue to monitor his vitals and let me know if there are any significant changes.
Speaker 2: Absolutely, Doctor. I'll keep you updated.

"""
llm(prompt)

'{ "Id": 1, "age": 65, "diagnosis": "Type 2 Diabetes", "gender": "Male", "name": "Mr. Anderson" } '

In [6]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Replicate(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="andreasjansson/llama-2-13b-chat-gguf:60ec5dda9ff9ee0b6f786c9d1157842e6ab3cc931139ad98fe99e08a35c5d4d4",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1, "jsonschema": "{\n \"type\": \"object\", \n \"properties\": {\n \"Id\" : {  \"type\" : \"integer\"}, \n \"name\" : {\n \"type\" : \"string\"}, \n \"age\" : {\n \"type\" : \"integer\"}, \n \"gender\" : {\n \"type\" : \"string\" }, \n \"diagnosis\" : {\n \"type\" : \"string\" }} }"},
)
prompt = """
User: A male patient Alex is 20 years old have headache. Tell me what is the patient name, age, gender, and diagnosis
Assistant:
"""
_ = llm.invoke(prompt)

{ "Id": 123456789, "age": 20, "diagnosis": "tension headache", "gender": "Male" ,"name": "Alex"}